# Stage A Results Notebook — vMF Time-Series Features

This notebook is **for reporting + visualization only**.

**Workflow**
1. Run the Stage A pipeline from the repo root:
   ```bash
   python run_vmf_pipeline.py
   ```
2. Confirm the output file exists:
   - `outputs/vmf_subject_features.csv`
3. Then use this notebook to:
   - inspect the extracted features
   - generate figures
   - summarize relationships with `attention`, `p_factor`, etc.

> Important: This notebook **does not** re-run the feature extraction pipeline.
> If you change feature logic, edit the Python files and re-run `run_vmf_pipeline.py`.


## 0. Setup

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# Repo root assumptions:
# - You open this notebook from notebooks/ directory
# - The pipeline output lives at ../outputs/vmf_subject_features.csv

OUTPUT_CSV = Path("..") / "outputs" / "vmf_subject_features.csv"
OUTPUT_CSV


In [ ]:
if not OUTPUT_CSV.exists():
    raise FileNotFoundError(
        f"Missing output file: {OUTPUT_CSV}\n\n"
        "Run Stage A first from the repo root:\n"
        "  python run_vmf_pipeline.py\n"
    )
print(f"[OK] Found: {OUTPUT_CSV}")


## 1. Load Stage A outputs

In [ ]:
df = pd.read_csv(OUTPUT_CSV)
print("Shape:", df.shape)
df.head()


In [ ]:
df.describe(include="all").T.head(30)


## 2. Quick sanity checks

In [ ]:
# Required columns (minimum expected)
required_cols = ["subject"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Identify likely target columns (may or may not exist)
targets = [c for c in ["attention", "p_factor", "internalizing", "externalizing"] if c in df.columns]
print("Targets found:", targets)

# Identify feature columns (numeric, excluding subject + targets)
exclude = set(["subject"]) | set(targets)
feature_cols = [c for c in df.columns if c not in exclude and pd.api.types.is_numeric_dtype(df[c])]

print(f"# features: {len(feature_cols)}")
print("Example features:", feature_cols[:10])


## 3. Missingness summary

In [ ]:
# Missingness per column (top 20)
miss = df.isna().mean().sort_values(ascending=False)
miss.head(20)


## 4. Distributions of key dynamic features

In [ ]:
# Choose a few "core" features to visualize if present
core_feats = [
    "entropy_mean", "entropy_std",
    "switch_rate",
    "vol_l1", "vol_l2",
    "maxp_mean", "maxp_std",
    "self_transition_rate",
    "trans_entropy_mean",
]
core_feats = [c for c in core_feats if c in df.columns]
core_feats


In [ ]:
for c in core_feats:
    x = df[c].dropna().to_numpy()
    plt.figure()
    plt.hist(x, bins=40)
    plt.title(f"Histogram: {c}")
    plt.xlabel(c)
    plt.ylabel("Count")
    plt.show()


## 5. Scatter plots: features vs targets

These plots help interpret which vMF dynamics relate to outcomes.


In [ ]:
def scatter_feature_vs_target(feature: str, target: str):
    sub = df[[feature, target]].dropna()
    if len(sub) < 5:
        print(f"[SKIP] Not enough data for {feature} vs {target}")
        return

    plt.figure()
    plt.scatter(sub[feature], sub[target])
    plt.xlabel(feature)
    plt.ylabel(target)
    plt.title(f"{feature} vs {target} (n={len(sub)})")
    plt.show()

# Try core features vs each target
for target in targets:
    for feature in core_feats:
        scatter_feature_vs_target(feature, target)


## 6. Correlations (numeric features)

This section computes correlations between:
- dynamic features
- and each target

We report the **top absolute correlations** for interpretability.


In [ ]:
numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
corr = df[numeric_cols].corr(numeric_only=True)
corr.shape


In [ ]:
def top_corr_with_target(target: str, k: int = 20):
    if target not in corr.columns:
        print(f"[SKIP] {target} not found in numeric columns.")
        return
    s = corr[target].drop(labels=[target]).dropna().abs().sort_values(ascending=False)
    top = s.head(k)
    out = pd.DataFrame({
        "feature": top.index,
        "abs_corr": top.values,
        "corr": [corr.loc[f, target] for f in top.index],
    })
    return out

for target in targets:
    print(f"\nTop correlations with {target}:")
    display(top_corr_with_target(target, k=15))


## 7. Simple baselines (optional)

`run_vmf_pipeline.py` already prints cross-validated performance.

This section provides a quick *in-notebook* baseline fit (train/test split) for sanity.
It is **not** a substitute for the script’s grouped cross-validation.


In [ ]:
from numpy.random import default_rng

def train_test_baseline(target: str, test_frac: float = 0.2, seed: int = 123):
    if target not in df.columns:
        print(f"[SKIP] target {target} not in df.")
        return

    sub = df.dropna(subset=[target]).copy()
    # Select numeric features only
    exclude = {"subject", target} | set([t for t in ["attention", "p_factor", "internalizing", "externalizing"] if t in sub.columns])
    feat_cols = [c for c in sub.columns if c not in exclude and pd.api.types.is_numeric_dtype(sub[c])]

    X = sub[feat_cols].to_numpy(dtype=float)
    y = sub[target].to_numpy(dtype=float)

    # Standardize features
    mu = X.mean(axis=0)
    sd = X.std(axis=0)
    sd = np.where(sd < 1e-12, 1.0, sd)
    Xz = (X - mu) / sd

    rng = default_rng(seed)
    idx = np.arange(len(y))
    rng.shuffle(idx)
    n_test = max(1, int(test_frac * len(y)))
    te = idx[:n_test]
    tr = idx[n_test:]

    # Ridge with intercept, no sklearn
    alpha = 10.0
    Xtr = np.column_stack([np.ones(len(tr)), Xz[tr]])
    Xte = np.column_stack([np.ones(len(te)), Xz[te]])
    p = Xtr.shape[1]
    I = np.eye(p); I[0,0] = 0.0
    coef = np.linalg.solve(Xtr.T @ Xtr + alpha*I, Xtr.T @ y[tr])
    yhat = Xte @ coef

    ss_res = float(np.sum((y[te] - yhat)**2))
    ss_tot = float(np.sum((y[te] - np.mean(y[te]))**2))
    r2 = 1.0 - ss_res/ss_tot if ss_tot > 1e-12 else 0.0

    plt.figure()
    plt.scatter(y[te], yhat)
    plt.xlabel(f"True {target} (test)")
    plt.ylabel(f"Predicted {target} (test)")
    plt.title(f"Quick ridge baseline (not grouped CV) — R2={r2:.3f}")
    plt.show()

    print(f"[Baseline] target={target}, n_test={len(te)}, R2={r2:.3f}")

for target in targets:
    train_test_baseline(target)


## 8. Notes for students

- If you want to change **how features are computed**, edit `vmf_features.py`, then re-run:
  ```bash
  python run_vmf_pipeline.py
  ```
- If you want to change **which files are loaded**, edit `config.py` or `vmf_dataset.py`.
- The notebook should remain focused on **plots and interpretation**, not the pipeline.
